## Websocket Server 

In [1]:
import asyncio
import os
import websockets
from re import search

class WebsocketServer:
    """
    Websocket server which can receive messages from any user.
    """
    
    def __init__(self, server_ip="$USER", server_port=8080):
        """
        Init of WebsocketServer. 
        Create dictionary for all connected clients with name and websocketobject and starts websocket server.
        server_ip (str): Either and IPv4 address or a name which can get resolved to an IP address.
        server_port (int): default port is 8080
        """
        self.server_port = server_port
        self.clients_connected = dict() # key:name, value:websocketobject
        self.server_ip = os.path.expandvars(server_ip)
        self.websocket_server = websockets.serve(self.connection_handler, self.server_ip, self.server_port)
        print("Init of WebsocketServer.")
        
    async def connection_handler(self, websocket, path):
        """
        Handles the communication patterns 1:all (broadcast) and 1:1 as well as the connected clients in a dictionary.
        websocket (websocket object): A websocket-server-protocol object that will be used for communication.
        path (str): Returns the path of uri of request.
        """
        first = True
        user_name = None
        
        try:
            async for msg in websocket:
                if first:
                    user_name = msg
                    print(f"Address: {websocket.remote_address} of User {user_name}")
                    self.clients_connected[user_name] = websocket 
                    print(f"All Clients connected: {self.clients_connected}\n")
                    first = False
                    continue
        
                receiver, message = msg.split(':', maxsplit=1)
                    
                if receiver == "*": 
                    for name, websocketobject in self.clients_connected.items(): 
                        if name != user_name:
                            await websocketobject.send(f"{user_name}:{message}") # send to all clients
                else:
                    if receiver in self.clients_connected: 
                        await self.clients_connected[receiver].send(f"{user_name}:{message}") # send to one client
                    else:
                        await websocket.send(f"Server: {receiver} does not exist. Try sending it to another User.")
                
        except websockets.ConnectionClosedError:
            pass
        
        finally:
            print(f"User {user_name} was successfully disconnected.")
            del self.clients_connected[user_name]
    

################ Start Websocket-Server ################
server = WebsocketServer(server_ip='kvanc.exercise')
asyncio.ensure_future(server.websocket_server)

Init of WebsocketServer.


<Task pending name='Task-1' coro=<_wrap_awaitable() running at /opt/conda/envs/kvanc/lib/python3.8/asyncio/tasks.py:677>>